In [5]:
from langchain.llms import GooglePalm
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["GOOGLE_PALM_KEY"]=os.getenv("GOOGLE_PALM_KEY")
api_key = os.getenv("GOOGLE_PALM_KEY") # get this free api key from https://makersuite.google.com/

In [ ]:
# print(os.getenv("GOOGLE_PALM_KEY"))

In [12]:
load_dotenv()
llm = GooglePalm(google_api_key=os.getenv("GOOGLE_PALM_KEY"), temperature=0.1)

c:\Users\10694387\OneDrive - LTIMindtree\Documents\Pytorch\LangChain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
poem = llm("Write a 4 line poem of my love")
print(poem)

My love for you is like a red, red rose
That's newly sprung in June:
My love is like the melody
That's sweetly played in tune.


In [18]:
essay = llm("write email requesting refund for electronic item")
print(essay)

Dear [Name of Customer Service Representative],

I am writing to request a refund for the [product name] that I purchased on [date]. I am not satisfied with the product because [explain why you are not satisfied].

I have attached a copy of my receipt and a picture of the product. I have also included a link to the product's listing on your website.

I would like to request a full refund for the product. I would also like to request that you send me a replacement product.

I have been a loyal customer of your company for many years, and I have never had a problem with a product before. I am disappointed that I am not satisfied with this product, but I am confident that you will be able to resolve this issue to my satisfaction.

Thank you for your time and attention to this matter.

Sincerely,
[Your Name]


In [19]:
a = llm("in which mutual fund i have to invest to get more returns")
print(a)

There are many factors to consider when choosing a mutual fund, including your risk tolerance, time horizon, and investment goals. However, some mutual funds are more likely to generate higher returns than others.

One type of mutual fund that has the potential to generate higher returns is a growth fund. Growth funds invest in stocks of companies that are expected to grow rapidly. These companies may be small, young companies or they may be established companies that are experiencing a period of rapid growth.

Growth funds can be riskier than other types of mutual funds, as the stocks they invest in are more volatile. However, they also have the potential to generate higher returns over the long term.

If you are looking for a mutual fund that has the potential to generate higher returns, a growth fund may be a good option for you. However, it is important to remember that there is no guarantee of returns, and you should only invest in a growth fund if you are willing to accept the ri

In [20]:
from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

In [21]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='codebasics_faqs.csv', source_column="prompt")

# Store the loaded data in the 'data' variable
data = loader.load()

In [26]:
embeddings = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the query for retrieval: "
)

No sentence-transformers model found with name hkunlp/instructor-large. Creating a new one with mean pooling.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /hkunlp/instructor-large/resolve/main/config.json (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1002)')))"), '(Request ID: 22a378c0-d1c2-4d64-96c8-f39b3d7c9be9)')

In [30]:
from langchain.embeddings import OllamaEmbeddings

ollama_emb = OllamaEmbeddings(
    model="llama2",
)

r2 = ollama_emb.embed_query("What is the second letter of Greek alphabet")

In [32]:
print(len(r2))

4096


In [34]:
e = ollama_emb.embed_query("What is your refund policy?")

In [35]:
len(e)

4096

In [37]:
type(e)

list

In [38]:
print(e[:5])

[-0.6700166463851929, 0.40010321140289307, 2.414700746536255, 0.1586562693119049, -3.446977138519287]


In [42]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=ollama_emb)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [43]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

c:\Users\10694387\OneDrive - LTIMindtree\Documents\Pytorch\LangChain\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Is there any prerequisite for taking this course?\nresponse: The only prerequisite is that you need to have a functional laptop with at least 4GB ram, internet connection and a thrill to learn data analysis.', metadata={'source': 'Is there any prerequisite for taking this course?', 'row': 35}),
 Document(page_content='prompt: Is there any prerequisite for taking this bootcamp ?\nresponse: Our bootcamp is specifically designed for beginners with no prior experience in this field. The only prerequisite is that you need to have a functional laptop with at least 4GB ram, an internet connection, and a thrill to learn data analys

In [63]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=[ "question"]
)

PROMPT.format(context="Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.",question="Do you provide job assistance ?")

'Given the following context and a question, generate an answer based on this context only.\nIn the answer try to provide as much text as possible from "response" section in the source document context without making much changes.\nIf the answer is not found in the context, kindly state "I don\'t know." Don\'t try to make up an answer.\n\nCONTEXT: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.\n\nQUESTION: Do you provide job assistance ?'

In [61]:
from langchain_core.prompts import PromptTemplate

# Instantiation using from_template (recommended)
prompt = PromptTemplate.from_template("Say {foo}")
prompt.format(foo="bar")

# Instantiation using initializer
prompt = PromptTemplate(template="Say {foo}",input_variables=["foo"])


In [69]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

response=chain.invoke({"input": "Do you provide job assistance ?"})

In [72]:
print(response['context'])

[Document(page_content='prompt: How can I use PowerBI on my Mac system?\nresponse: Hi\n\nYou can use VirtualBox to create a virtual machine and install Windows on it. This will allow you to run Power BI and Excel on your Mac.\n\nIf you\'re not familiar with setting up a virtual machine, there are many resources available on YouTube that can guide you through the process. Simply search for "installing virtual machines" and you\'ll find plenty of helpful videos.\n\nBest of luck with your studies!', metadata={'source': 'How can I use PowerBI on my Mac system?', 'row': 44}), Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}), Document(page_content='prompt: Is there any prerequisite for taking this course?\nresp

In [71]:
print(response['answer'])


response: Yes, we provide job assistance to our students. We help them with resume and interview preparation, and we also refer them to potential employers.


In [67]:
# from langchain.prompts import PromptTemplate

# prompt_template = """Given the following context and a question, generate an answer based on this context only.
# In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
# If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

# CONTEXT: {context}

# QUESTION: {question}"""


# PROMPT = PromptTemplate(
#     template=prompt_template, input_variables=[ "context", "question"]
# )
# chain_type_kwargs = {"prompt": PROMPT}


# from langchain.chains import RetrievalQA

# chain = RetrievalQA.from_chain_type(llm=llm,
#                             chain_type="stuff",
#                             retriever=retriever,
#                             input_key="query",
#                             return_source_documents=False,
#                             chain_type_kwargs=chain_type_kwargs)

In [68]:
chain('Do you provide job assistance ?')

{'query': 'Do you provide job assistance ?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'}